In [65]:
import os
from sqlalchemy import create_engine
import pandas as pd
from dotenv import load_dotenv
from icecream import ic
import json
from datetime import datetime
from copy import deepcopy
load_dotenv()

True

## load variables - invasive species
- data folder
- cleaned_df - for only invasive species
- json template
- 

In [66]:
data_folder = os.path.join(os.getcwd(),'data')
cleaned_csv = os.path.join(data_folder, 'final_range_data.csv')
dff = pd.read_csv(cleaned_csv)

#lambda function to filter for invasives
#for each row, is it a list/str, then is 'invasives' in it, if so, return the df.
cleaned_df = dff[
    dff["land_use_selection-intervention"]
    .apply(lambda x: isinstance(x, (list, str)) and " invasive_removal" in x)
]

#no ivasive species datasets

TEMPLATE_FILE = os.path.join(data_folder,"halfmoon_json_template.json")
with open(TEMPLATE_FILE, "r") as f:
    template = json.load(f)



## check invasives count

In [67]:
x = dff["land_use_selection-intervention"]
count = 0
for idx, i in enumerate(list(x)):
    if 'invasive_removal' in i:
        count += 1
        ic(i, count)

ic('invasive_removal count',len(cleaned_df))

ic| i: 'halfmoons water_points invasive_removal', count: 1
ic| 'invasive_removal count', len(cleaned_df): 1


('invasive_removal count', 1)

## enumerators

In [68]:
#sorted(list(dff['basic_info-name'].unique()))
##empty name and key dictionary
names_dict = {}
for idx, row in dff.iterrows():
    #ic({row["KEY"], row["basic_info-name"]})
    names_dict[row["KEY"]] = row["basic_info-name"]
#ic(names_dict)
name_uname_dict = {
    "Zakariye haji ali": "abdullahi.hassan@mardo.org", 
    "Issack Idow Ibrahim": "abdullahi.hassan@mardo.org",
    "Abdullahi Hassan Adan": "abdullahi.hassan@mardo.org",
    "Abdikheir Mohamed": "mukhtar.yusuf23",
    "Abdikheir Mohamed Ali": "mukhtar.yusuf23",
    "Abdikheir Mohamed Dahir": "mukhtar.yusuf23",
    "Ali": "mukhtar.yusuf23",
    "Ali Adan": "mukhtar.yusuf23",
    "Ali Aden": "mukhtar.yusuf23",
    "Fathi": "mukhtar.yusuf23", 
    "Fathi Abdirashid": "mukhtar.yusuf23",
    "Ahmed Omar Hassan": "AhmedOmar",
    "Maria Abdirahman Ali": "AhmedOmar",
    "ABSHIR ABDULLAH ALI": "Ahil",
    "Ahmed Ibrahim": "JubaFoundation",
    "Ahmed Ibrahim Mohamed": "JubaFoundation",
    "Abdiaziz Adan Hassa": "guudow",
    "Abdiaziz Adan Hassan": "guudow",
    "Abdi Hassan Adan": "AbdiHassan",
    "Abdullahi": "sharif1234",
    "Abdirashid Sheikh Mohamed": "NRMO",
    "Abdullahi Sharif": "sharif1234",
    "Abdullahi Sharif LLG": "sharif1234",
    "Abdullahi Sharif Noor LLG": "sharif1234",
    "Abdijalil": "Csxaashi",
    "Abdijalil said": "Csxaashi",
    "Abdikarim Mohamed Ismail": "samandoulgou",
    "Abdikarin Mohamed Ismail": "samandoulgou",
    "Hassan Omar Khadhib": "Khadhib",
    "Hassan Omar Khadhibr": "Khadhib",
    "Jamal Ali": "JamalAli",
    "Jamal Ali Tagal": "JamalAli",
    "Mohamud Abdullahi Ali": "Mohamud",
    "Moulid Abdullahi Abdi": "Moulidwadani",
    "Muktar Mohamednoor Rage": "Mrage1997",
    "Osman Ebey Omar": "iibey",
    "Yusuf Mahi munye": "Mcnbdn617660",
    "abdullahi Sharif LLG": "sharif1234",
    "osman Ebey Omar": "iibey",
}

## match and replace names with usernames
for k, v in names_dict.items():
    for key, value in name_uname_dict.items():
        if v == key:
            names_dict[k] = value
#ic(names_dict)

##enumerator dictionary
enumerator = {
        "id": 3726,
        "first_name": "First",
        "last_name": "Last",
        "gender": "Male",
        "age_category": "18 - 35",
        "phone_number": "0e15704046",
        "email": "test_mail@gmail.com",
        "country": "test",
        "organization": "Test",
        "username": "test",
        "type": "ENUMERATOR"
    }

enums = []
#create enumerator jsons
for key, value in names_dict.items():
    e = deepcopy(enumerator)
    e['username'] = value
    e['key'] = key
    enums.append(e)

#ic(enums)

## farming entity

In [69]:
farming_entity_list = []
for idx, i in enumerate(cleaned_df.to_dict(orient='records'), start=1):
    farming_entity = {}
    #ic(i['basic_info-organisation_name'])
    farming_entity['id'] = idx
    farming_entity['first_name'] = i['basic_info-organisation_name']
    farming_entity['middle_name'] = i['basic_info-organisation_name']
    farming_entity['last_name'] = i['basic_info-organisation_name']
    farming_entity['organization'] = 'INSTITUTION'
    farming_entity['key'] = i['KEY']
    
    farming_entity_list.append(
        {
            "farmingEntity": farming_entity, 
            'key': i['KEY']
        }
    )
    #ic(farming_entity_list)
    

## plot-invasives_details
- read df to dictionary, for matching keys, extract df values
### plot-points
- parse the geoshape column from cleaned df
- split and extract the values from each i
- append to json keys

## crops
- no need for crop details as repeat is not there
 

In [70]:
import uuid
import copy
import re

def parse_geoshape(geoshape_str):
    #parse a geoshape string into a list of point dictionaries
    points = []
    parts = geoshape_str.split(';')

    for part in parts:
        part = part.strip()
        if not part:
            continue

        lat, lon, alt, acc = map(float, part.split(' '))

        points.append({
            "longitude": lon,
            "latitude": lat,
            "altitude": alt,
            "accuracy": acc
        })

    return points

plot_list = []
point_id = 1

records = cleaned_df.to_dict(orient='records')

count = 0
#iterate over every i to capture plot details
for idx, i in enumerate(records):
    plot_id = idx + 1
    count += 1

    # ---- BUILD PLOT DETAILS ----
    template_cp = copy.deepcopy(template[0])
    plot_details = template_cp['plot']

    plot_details['id'] = plot_id
    plot_details['key'] = i['KEY']
    plot_details['name'] = "uuid:" + str(uuid.uuid4())
    plot_details['calculated_size'] = i['area_ha']
    plot_details['photo_url'] = i['plot_details-photo']
    plot_details['farmingEntityId'] = farming_entity['id']
    plot_details['livestock_allowed'] = False if i['land_use_selection-land_use'] == 'agriculture' else True
    plot_details['conservation_area'] = False
    
    #no need for crop details 
        
    """--------------------------------GEOSHAPE PARSING ----------------------------------"""
    geoshape_str = i['plot_details-polygon']
    ic(geoshape_str, i["KEY"])

    parsed_points = parse_geoshape(geoshape_str)

    points_list = []
    for p in parsed_points:
        points_list.append({
            "id": point_id,
            "plotId": plot_id,
            "longitude": p["longitude"],
            "latitude": p["latitude"],
            "altitude": p["altitude"],
            "accuracy": p["accuracy"]
        })
        point_id += 1


    """--------------------------------ADMINISTRATIVE DETAILS ----------------------------------"""
    administrative = plot_details['subCounty']
    administrative['subcounty_name'] = i['geography-district_name']
    administrative['county']['county_name'] = i['geography-region_name']


    #update plot_details with number of points and administrative info
    plot_details['points'] = points_list
    plot_details['subCounty'] = administrative
    
    # del plot_details['points']
    # del plot_details['subCounty']
    general_plot = {
        "plot":plot_details, 
        'key':i['KEY']
    }
    #ic(i['KEY'])
    plot_list.append(
        general_plot
    )


ic| geoshape_str: '6.6514281 46.9264623 309.0 7.833;6.6514948 46.926457 316.0 2.38;6.6516772 46.926446 318.0 2.316;6.651778 46.9264377 317.0 2.38;6.651948 46.9264147 319.0 2.28;6.6521508 46.9263863 321.0 2.38;6.6523469 46.9263628 321.0 2.38;6.6524931 46.9263556 320.0 2.316;6.6526859 46.9263277 321.0 2.316;6.6528936 46.9263071 321.0 2.38;6.6530636 46.9262831 322.0 2.316;6.6532716 46.9262469 321.0 2.316;6.6534731 46.9262218 321.0 2.38;6.653641 46.9261973 321.0 2.08;6.6538445 46.9261723 319.0 2.28;6.6540471 46.9261441 319.0 2.38;6.6542244 46.9261217 318.0 2.316;6.6544386 46.9260928 319.0 2.38;6.6546394 46.9260635 319.0 2.316;6.6548222 46.9260321 319.0 2.38;6.6550299 46.926028 318.0 2.38;6.6552391 46.9259801 318.0 2.233;6.6554134 46.925953 317.0 2.38;6.6554412 46.9260398 319.0 2.233;6.6553898 46.9262381 319.0 2.28;6.655322 46.9264254 317.0 2.233;6.6552517 46.9266383 318.0 2.316;6.6551833 46.9268638 318.0 2.38;6.655136 46.9270228 318.0 2.38;6.655067 46.9272242 317.0 2.316;6.6549976 46.92744

In [71]:
with open(os.path.join(data_folder, 'to_delete.json'), 'w') as f:
    json.dump(plot_list, f, indent=4)



## project

In [72]:
projects_list = []

names =[]
for row in cleaned_df.to_dict(orient='records'):
    name = row['land_use_selection-project_name']
    

    if name == 'terra':
        name = name.upper()
        id = 131
    elif name == 'brcis3':
        name = 'BRCiS III'
        id = 93
    names.extend([name])
    projects_list.append({
        "project": {
            "id": id,
            "project_name": name
        },
        "key": row['KEY']
    })

ic(set(names))

ic| set(names): {'BRCiS III'}


{'BRCiS III'}

## Invasives 
missing 
- herbacious cover
- who manages >> used who established
- erosion type
- is_intervention_effective
- vertical_spacing
- horizontal_spacing

In [ ]:
invasivs_list = []


for idx, c in enumerate(records):

    #if c['KEY'] == "uuid:1975a725-83a5-4bdd-8305-35ac65b2cce9":
    """-----------------------------------INVASIVES CHILD DETAILS-------------------------------------"""
    #no invasve child detail

    """-----------------------------------INVASIVES DETAILS-------------------------------------"""

    invasives_details = {
        "key":c["KEY"],
        "id": idx,
        "removal_date": datetime.strptime(c['invasive_removal-removal_date'], "%d/%m/%Y").strftime("%Y-%m"),
        "removal_methods": c["invasive_removal-removal_methods"].split(),
        "other_removal_methods": "",
        "photo_url": "", #no photo column
        "comments": f"Invasive_species NONE, iremoval_severity {c["invasive_removal-severity"].upper()}",
        "rangelandEntryId": 8,
        "grassCurrentStatus": [],
        "treeCurrentStatus": []
    }

    invasivs_list.append(invasives_details)
    

ic(invasivs_list)
iremoval_list = [{"iremoval": [i], "key" :i["key"]} for i in invasivs_list]


ic| invasivs_list: [{'comments': 'Invasive_species NONE, iremoval_severity LESS_25',
                     'grassCurrentStatus': [],
                     'id': 0,
                     'key': 'uuid:48f049ba-b292-4c09-a914-7667d1158a10',
                     'other_removal_methods': '',
                     'photo_url': 'JPEG_5cc8d00f-cbd5-41a2-a868-374783263033_3612000906371608906.jpg',
                     'rangelandEntryId': 8,
                     'removal_date': '2025-05',
                     'removal_methods': ['manual'],
                     'treeCurrentStatus': []}]


In [90]:
with open(os.path.join(data_folder, 'to_delete.json'), 'w') as f:
    json.dump(iremoval_list, f, indent=4)

## merge components into one json

In [91]:
# component lists
lists = [farming_entity_list, iremoval_list,plot_list, projects_list]

# The output dictionary
merged = {}

for lst in lists:
    ic(len(lst))
    for item in lst:
        k = item['key']

        #check if already updated
        if k not in merged:
            merged[k] = {}
            
        #merge the item into the existing dictionary
        merged[k].update(item)

final = list(merged.values())
for i in final:
    #match enumerators
    for l in enums:
        if i['key'] == l['key']:
            i['enumerator'] = l


    i["boma"] = []
    i["cgrazing"] = []
    i["microcatchment"]= []
    i["seedbank"] = []
    i["seeding"] = []
    i["waterpoint"] = []
    i["completed"] = True
    i["rangeModuleId"] = 1
    i["currentStep"]= "DONE"
    i["moduleId"] = 6
    i["is_revisit"] = False
    #get the date_collected
    for j in records:
        if i["key"] == j["KEY"]:
            i["date_collected"] = datetime.strptime(j['basic_info-date'], "%d/%m/%Y").strftime("%Y-%m-%d")
    #match project id
    for x in projects_list:
        if i['key'] == x['key']:
            i['projectId'] = x['project']['id']
    #match plot id
    for k in plot_list:
        if i['key'] == k['key']:
            i['plotId'] = k['plot']['id']
    #match farming_entity_id
    for h in farming_entity_list:
        if i['key'] == h['key']:
            i['farmingEntityId'] = h['farmingEntity']['id']
    #match enumerator id
    for e in enums:
        if i['key'] == e['key']:
            i['enumeratorId'] = e['id']


ic(len(final))


ic| len(lst): 1
ic| len(lst): 1
ic| len(lst): 1
ic| len(lst): 1
ic| len(final): 1


1

In [92]:
with open(os.path.join(data_folder, 'to_delete.json'), 'w') as f:
    json.dump(final, f, indent=4)

## clean up keys

In [93]:
#clean up keys
def remove_keys(obj, key="key"):
    if isinstance(obj, dict):
        obj.pop(key, None)
        for v in obj.values():
            remove_keys(v, key)
    elif isinstance(obj, list):
        for v in obj:
            remove_keys(v, key)
    return obj

finaal = [x for x in final if x['key'] == 'uuid:b4a2d545-66fe-4d23-825b-41025149b460']
finaal = [remove_keys(i) for i in final]

#add id
for idx, c in enumerate(finaal):
    c['id'] = idx + 1



In [94]:
with open(os.path.join(data_folder, 'to_delete.json'), 'w') as f:
    json.dump(finaal, f, indent=4)